In [1]:
import numpy as np
from matplotlib import pyplot

In [2]:
def Crop(M_arr,P): #croppa una matrice M di pattern=righe e restituisce un M_mat (pattern=matrice)
                   #COSTRUISCE ANCHE M_MAT
    M_inv = np.zeros((28,28,P))
    for i in range(0,P):
        M_inv[:,:,i] = M_arr[i].reshape(28,28) #trasforma una matrice in un array di matrici da M a M_mat

    return M_inv[3:23,3:23,:] #M_mat

In [3]:
def Input_mnist(P):
    import numpy as np
    from keras.datasets import mnist
                                            # loading
    (X, y), (k,h) = mnist.load_data()
                                            # Define patterns
    M_arr = np.zeros((P,28*28))
    l    = 0
    flag = 0
    for i in range(0,P):
        for j in range(np.size(y)):
            if y[j] == i:
                if flag == 0:
                    m = j
                    M_arr[l,:] = np.ravel(X[j,:,:]) 
                    flag = 1
                    l += 1
        flag = 0
        
    for i in range(0,P):                  #SCALA DI GRIGI
        M_arr[i] /= np.max(M_arr[i])
    mask = M_arr<=0.5
    M_arr[mask] = -1
    M_arr[~mask] = 1
    
    M_mat = Crop(M_arr,P)
    
    M_arr = np.zeros((P,20*20))            #croppata
    for i in range(0,P):
        M_arr[i,:] = np.ravel(M_mat[:,:,i]) 
    return M_arr                           #M_mat è la matrici è la matrice che ha come pattern matrici

#M_mat, M = Input_mnist(P)
#plot_img(M_mat,P)

In [4]:
def Crop_selection(M_arr,K=[]):
    
    M_inv = np.zeros((28,28,len(K)))
    for i in range(len(K)):
        M_inv[:,:,i] = M_arr[i].reshape(28,28)
        
    return M_inv[3:23,3:23,:]

In [5]:
def Input_mnist_selection(K=[]):
    import numpy as np
    from keras.datasets import mnist
    # loading
    (X, y), (k,h) = mnist.load_data()
    # Define patterns
    M_arr = np.zeros((len(K),28*28))
    l    = 0
    flag = 0
    for i in K:
        for j in range(np.size(y)):
            if y[j] == i:
                if flag == 0:
                    m = j
                    M_arr[l,:] = np.ravel(X[j,:,:]) 
                    flag = 1
                    l += 1
        flag = 0
    for i in range(len(K)):
        M_arr[i] /= np.max(M_arr[i])
    mask = M_arr<=0.5
    M_arr[mask] = -1
    M_arr[~mask] = 1
    
    M_mat = Crop_selection(M_arr,K)
    M_arr = np.zeros((len(K),20*20))
    for i in range(0,len(K)):
        M_arr[i,:] = np.ravel(M_mat[:,:,i]) 
    return M_arr

In [6]:
def Plot(M_arr,P):
    M_inv = np.zeros((20,20,P))
    for i in range(0,P):
        M_inv[:,:,i] = M_arr[i].reshape(20,20)
        pyplot.imshow(M_inv[:,:,i], cmap=pyplot.get_cmap('gray'))
        pyplot.show()
    #return M_inv

In [7]:
def Plot_selection(M_arr,K=[]):
    M_inv = np.zeros((20,20,len(K)))
    for i in range(0,len(K)):
        M_inv[:,:,i] = M_arr[i].reshape(20,20)
        pyplot.imshow(M_inv[:,:,i], cmap=pyplot.get_cmap('gray'))
        pyplot.show()
    #return M_inv

In [8]:
def J(M_arr):
    
    matrix_dim=M_arr.shape
    J=np.zeros((matrix_dim[1],matrix_dim[1]))

    for i in range(0,matrix_dim[1]):
        for j in range(0,matrix_dim[1]):
            if j!=i:
                J[i,j]=np.sum([M_arr[a,i]*M_arr[a,j] for a in range(0,matrix_dim[0])])/matrix_dim[1]
    return J

In [9]:
# def J_final(M,n,P):
#     matrix_dim=M.shape
#     J_final=np.zeros((matrix_dim[1],matrix_dim[1]))
#     M=M.reshape((P,n,n))
    
#     matrix_dim=M.shape
#     for i in range(matrix_dim[1]):
#         for j in range(matrix_dim[1]):
#             for k in range(matrix_dim[1]):
#                 for l in range(matrix_dim[1]):
#                     dist=Distanza(i,j,k,l)
#                     if i!=j and dist<=2 and dist!=0:
#                         J_final[i,j]=np.sum([M[a,i,j]*M[a,k,l] for a in range(P)])
#     return J_final

In [10]:
def Distanza(i,j,k,l):
    dist=np.sqrt((k-i)*(k-i)+(l-j)*(l-j))
    return dist

def Interazione(J,n,d):
    
    for i in range(n):
            for j in range(n):
                for k in range(n):
                    for l in range(n):
                        dist=Distanza(i,j,k,l)
                        if dist>=d:
                            a=i*n+j
                            b=k*n+l
                            J[a,b]=0
    return J

In [11]:
def Wrong_patterns(q,M_arr):
    from numpy import random

    Y=np.zeros((M_arr.shape))

    for i in range(0,M_arr.shape[0]):
        x=M_arr[i,:]

        rand=np.random.random_sample()
        for j in range(0,M_arr.shape[1]):
            rand=np.random.random_sample()
            if rand<=q:
                Y[i,j]=x[j]
            else:
                Y[i,j]=-x[j]
    return Y

In [12]:
def update_y(J,y,M_arr,iter_max=100):
    import numpy as np
    flag = 1
    t    = 0 
    
                            # Update
    y_old    = np.zeros(len(y))
    y_old[:] = y[:]
    while(flag and t <= iter_max):
        #print('old y:', y_old)
        for i in range(J.shape[1]):
            y[i] = np.sign(np.dot(J[i,:],y))
            if (y[i]==0):
                y[i] = 1
            #print(y)
        t += 1
        k = 0
        for j in range(0,len(y)):
            if(y[j] == y_old[j]):
                k += 1
        if k==len(y):
            flag = 0
        else:
            y_old[:] = y[:]
                
                                # Pattern of y updated
    P_a = -1
    if (flag == 0):
        for i in range(M_arr.shape[0]):
            k = 0
            for j in range(0,len(y)):
                if(y[j] == M_arr[i,j]):
                    k += 1
            if k == len(y):
                P_a = i
    #print("Final y:")
    #print(y)
    return P_a,t

In [13]:
P=4
K=[0,1,2,3,4,5,6,7,8,9]
M_arr=Input_mnist_selection(K)
#M_arr=Input_mnist(P)
M_arr = np.array(M_arr,dtype=np.float64)
J_matrix=J(M_arr)
A=np.zeros((400,400))
A[:,:]=J_matrix[:,:]
J_corretta=Interazione(A,20)
print('J: \n', J_matrix)
print(J_corretta)

TypeError: Interazione() missing 1 required positional argument: 'd'

In [ ]:
Y_1=Wrong_patterns(0.9,M_arr)
Y_2=np.zeros(Y_1.shape)
Y_2[:,:]=Y_1[:,:]
#Matrix_mnist(Y,P)
Plot_selection(Y_1,K)

In [ ]:
for j in range(0,len(K)):
        pattern_giusto, iterazione= update_y(J_matrix,Y_1[j],M_arr) 
        if pattern_giusto==-1:
            print('no pattern detected in:', iterazione, 'iterations \n')
        else:
            print('pattern detected: ', pattern_giusto, 'in ', iterazione, 'iterations \n')
Plot_selection(Y_1,K)

In [ ]:
for j in range(0,len(K)):
        pattern_giusto, iterazione= update_y(J_corretta,Y_2[j],M_arr) 
        if pattern_giusto==-1:
            print('no pattern detected in:', iterazione, 'iterations \n')
        else:
            print('pattern detected: ', pattern_giusto, 'in ', iterazione, 'iterations \n')
Plot_selection(Y_2,K)